In [5]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.python.keras.initializers import Constant

GloVe embedding data (Wikipedia 2014 + Gigaword 5) can be found at:
http://nlp.stanford.edu/data/glove.6B.zip

20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html

In [6]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

First, build index mapping words in the embeddings set to their embedding vector

In [10]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400001 word vectors.


Second, prepare text samples and their labels

In [12]:
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)
                
print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


Finally, vectorize the text samples into a 2D integer tensor

In [13]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 174074 unique tokens.


In [14]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


Split the data into a training set and a validation set

In [15]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

Prepare embedding matrix

In [16]:
print('Preparing embedding matrix.')

num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


Load pre-trained word embeddings into an Embedding layer, set trainable = False so as to keep the embeddings fixed

In [17]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Train a 1D convnet with global maxpooling

In [18]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Instructions for updating:
Colocations handled automatically by placer.
Train on 15998 samples, validate on 3999 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
15998/15998 [==============================] - 131s 8ms/sample - loss: 2.4392 - acc: 0.2085 - val_loss: 1.8008 - val_acc: 0.3803
Epoch 2/10
15998/15998 [==============================] - 132s 8ms/sample - loss: 1.5754 - acc: 0.4557 - val_loss: 1.3577 - val_acc: 0.5274
Epoch 3/10
15998/15998 [==============================] - 131s 8ms/sample - loss: 1.2086 - acc: 0.5821 - val_loss: 1.2982 - val_acc: 0.5406
Epoch 4/10
15998/15998 [==============================] - 138s 9ms/sample - loss: 0.9979 - acc: 0.6553 - val_loss: 1.0832 - val_acc: 0.6274
Epoch 5/10
15998/15998 [==============================] - 136s 9ms/sample - loss: 0.8351 - acc: 0.7143 - val_loss: 1.1637 - val_acc: 0.6217
Epoch 6/10
15998/15998 [=============================

Fetch the Keras session and save the model

In [25]:
export_path = 'D:/PythonProjects/Machine-Learning-Distributed-Systems/tmp/news/1'
tf.saved_model.simple_save(
    tf.keras.backend.get_session(),
    export_path,
    inputs={'document': model.input},
    outputs={t.name:t for t in model.outputs})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: D:/PythonProjects/Machine-Learning-Distributed-Systems/tmp/news/1\saved_model.pb


In [26]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['document'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 1000)
        name: input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1/Softmax:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 20)
        name: dense_1/Softmax:0
  Method name is: tensorflow/serving/predict


In [27]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": x_train[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ...  1276, 1, 157, 4, 2917, 2, 53, 132, 1780, 1, 1781]]}


In [37]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/news:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
for i in range(0,3):
  print("predicted: {}, real: {}".format(np.argmax(predictions[i]), np.argmax(y_train[i])))

predicted: 12, real: 12
predicted: 15, real: 15
predicted: 9, real: 9
